In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, Conv2D, BatchNormalization, LeakyReLU, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar100

# Tải dữ liệu CIFAR-100
(x_train, _), (_, _) = cifar100.load_data()
x_train = (x_train.astype("float32") - 127.5) / 127.5  # Chuẩn hóa [-1,1]
BUFFER_SIZE = x_train.shape[0]
BATCH_SIZE = 128
LATENT_DIM = 128  # Kích thước noise vector

# Dataset pipeline
dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

2025-03-06 21:59:48.118594: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2025-03-06 21:59:48.118621: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-03-06 21:59:48.118629: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-03-06 21:59:48.118649: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-06 21:59:48.118664: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
def build_generator():
    model = tf.keras.Sequential([
        Dense(8 * 8 * 256, use_bias=False, input_shape=(LATENT_DIM,)),
        BatchNormalization(),
        LeakyReLU(),
        Reshape((8, 8, 256)),  # Tạo feature map 8x8

        Conv2DTranspose(128, (5,5), strides=(2,2), padding="same", use_bias=False),
        BatchNormalization(),
        LeakyReLU(),

        Conv2DTranspose(64, (5,5), strides=(2,2), padding="same", use_bias=False),
        BatchNormalization(),
        LeakyReLU(),

        Conv2DTranspose(3, (5,5), strides=(1,1), padding="same", activation="tanh")  # Đầu ra 32x32x3
    ])
    return model

generator = build_generator()
generator.summary()

/Users/hiepnq/conda/envs/learn-ml/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16384)          │     2,097,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16384)          │        65,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 16, 16, 128)    │       819,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 64)     │       204,800 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 32, 32, 3)      │         4,803 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,192,259 (12.18 MB)

 Trainable params: 3,159,107 (12.05 MB)

 Non-trainable params: 33,152 (129.50 KB)

In [3]:
def build_discriminator():
    model = tf.keras.Sequential([
        Conv2D(64, (5,5), strides=(2,2), padding="same", input_shape=(32,32,3)),
        LeakyReLU(0.2),
        Dropout(0.3),

        Conv2D(128, (5,5), strides=(2,2), padding="same"),
        LeakyReLU(0.2),
        Dropout(0.3),

        Flatten(),
        Dense(1, activation="sigmoid")  # Output 0 (fake) hoặc 1 (real)
    ])
    return model

discriminator = build_discriminator()
discriminator.summary()

/Users/hiepnq/conda/envs/learn-ml/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 16, 16, 64)     │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 8, 8, 128)      │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 217,985 (851.50 KB)

 Trainable params: 217,985 (851.50 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
gen_optimizer = Adam(learning_rate=2e-4, beta_1=0.5)
disc_optimizer = Adam(learning_rate=2e-4, beta_1=0.5)

In [5]:
@tf.function
def train_step(real_images):
    batch_size = tf.shape(real_images)[0]
    noise = tf.random.normal([batch_size, LATENT_DIM])

    # Huấn luyện Discriminator
    with tf.GradientTape() as disc_tape:
        fake_images = generator(noise, training=True)
        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(fake_images, training=True)

        real_loss = loss_fn(tf.ones_like(real_output), real_output)
        fake_loss = loss_fn(tf.zeros_like(fake_output), fake_output)
        disc_loss = real_loss + fake_loss

    gradients_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    disc_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))

    # Huấn luyện Generator
    with tf.GradientTape() as gen_tape:
        fake_images = generator(noise, training=True)  # Sinh ảnh giả mới
        fake_output = discriminator(fake_images, training=True)
        gen_loss = loss_fn(tf.ones_like(fake_output), fake_output)

    gradients_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gen_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))

    return gen_loss, disc_loss

In [6]:
import os
EPOCHS = 200
SAVE_DIR = "./gan_cifar100"
os.makedirs(SAVE_DIR, exist_ok=True)

def generate_and_save_images(epoch):
    noise = tf.random.normal([16, LATENT_DIM])
    generated_images = generator(noise, training=False)
    generated_images = (generated_images + 1) / 2  # Chuyển về [0,1] để hiển thị

    fig, axs = plt.subplots(4, 4, figsize=(6,6))
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(generated_images[i*4 + j])
            axs[i, j].axis("off")
    plt.savefig(f"{SAVE_DIR}/epoch_{epoch}.png")
    plt.show()

for epoch in range(EPOCHS):
    for real_images in dataset:
        gen_loss, disc_loss = train_step(real_images)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: G Loss: {gen_loss.numpy():.4f}, D Loss: {disc_loss.numpy():.4f}")
        generate_and_save_images(epoch)

2025-03-06 21:59:59.752315: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-03-06 21:59:59.755929: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


KeyboardInterrupt: 